Mounting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Downloading BART and Tokenizer in Google Drive
(skip this step after downloading it the first time)

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration
import os

# Load pre-trained BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Specify the path in your Google Drive
model_dir = '/content/drive/MyDrive/bart-model'

# Save the model and tokenizer
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

print("Model and tokenizer saved to Google Drive.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Model and tokenizer saved to Google Drive.


Running the model

In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the model and tokenizer from Google Drive
model_dir = '/content/drive/MyDrive/bart-model'
tokenizer = BartTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

# Input text
text = '''
The global smartphone market experienced a significant shift in 2023,
with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year.
This was primarily due to supply chain disruptions, inflationary pressures, and geopolitical tensions.
In contrast, Apple's market share grew from 23.4% in 2022 to 26.5% in 2023, driven by strong demand for the iPhone 14 series,
particularly in North America and Europe. Meanwhile, Samsung's market share slightly decreased from 19.1% to 18.7%,
despite launching the Galaxy S23 and Galaxy Z Fold 5. Xiaomi retained its third position globally,
with an 11.8% market share, but also saw a 10% drop in shipments, largely due to a sluggish market in India.
Additionally, 5G-enabled devices accounted for 70% of total smartphone shipments, up from 57% in 2022,
reflecting increased consumer demand for faster mobile internet speeds. Analysts predict a rebound in 2024,
with smartphone shipments expected to grow by 4.3%, as supply chain constraints ease and
new innovations such as foldable screens and AI-powered features become more mainstream.

'''
# Encode the input text
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

The global smartphone market experienced a significant shift in 2023, with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year. 5G-enabled devices accounted for 70% of total smartphone shipments, up from 57% in 2022,reflecting increased consumer demand for faster mobile internet speeds. Analysts predict a rebound in smartphone shipments in the coming year, with shipments expected to grow by 4.3%.


**ROUGE Score**

Installing the libraries

In [ ]:
!pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e85b41aa27ca50408d02328bac085e8fb9e88a2bf6c8dcdb366a88c9ad60f60c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


Evaluating ROUGE scores

In [ ]:
import evaluate

# Initialize the ROUGE metric
rouge = evaluate.load("rouge")

# Original summary and generated summary
original_summary = '''
The global smartphone market experienced a significant shift in 2023,
with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year.
This was primarily due to supply chain disruptions, inflationary pressures, and geopolitical tensions.
In contrast, Apple's market share grew from 23.4% in 2022 to 26.5% in 2023, driven by strong demand for the iPhone 14 series,
particularly in North America and Europe. Meanwhile, Samsung's market share slightly decreased from 19.1% to 18.7%,
despite launching the Galaxy S23 and Galaxy Z Fold 5. Xiaomi retained its third position globally,
with an 11.8% market share, but also saw a 10% drop in shipments, largely due to a sluggish market in India.
Additionally, 5G-enabled devices accounted for 70% of total smartphone shipments, up from 57% in 2022,
reflecting increased consumer demand for faster mobile internet speeds. Analysts predict a rebound in 2024,
with smartphone shipments expected to grow by 4.3%, as supply chain constraints ease and
new innovations such as foldable screens and AI-powered features become more mainstream.

'''
generated_summary = '''
The global smartphone market experienced a significant shift in 2023,
with total shipments reaching 1.21 billion units, a decline of 7.1% compared to the previous year.
5G-enabled devices accounted for 70% of total smartphone shipments,
up from 57% in 2022,reflecting increased consumer demand for faster mobile internet speeds.
Analysts predict a rebound in smartphone shipments in the coming year, with shipments expected to grow by 4.3%
'''

# Calculate ROUGE scores
results = rouge.compute(predictions=[generated_summary], references=[original_summary])

# Display the results
print("ROUGE-1:", results["rouge1"])
print("ROUGE-2:", results["rouge2"])
print("ROUGE-L:", results["rougeL"])

ROUGE-1: 0.5433070866141733
ROUGE-2: 0.4920634920634921
ROUGE-L: 0.5118110236220472


**Gradio**

Installing the required libraries

In [ ]:
pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 690.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130

Summarization Function

In [6]:
def summarize_text_bart(text):
    # Preprocess the text (tokenize and encode)
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Setting Up Gradio Interface

In [7]:
import gradio as gr

# Create the Gradio interface
interface = gr.Interface(fn=summarize_text_bart,
                         inputs="text",
                         outputs="text",
                         title="BART Text Summarization",
                         description="Summarize text using the BART model. Enter text in the input box and get the summary below.")

# Launch the Gradio interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd37c83eff67ab0929.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
